### Test the VGG16 model with added layers on CIFAR-10

The code in this notebook was based off these tutorials: 

https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

https://towardsdatascience.com/transfer-learning-with-convolutional-neural-networks-in-pytorch-dd09190245ce

In [1]:
#from models.vgg import VGG16_model
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


Instantiate the model with 10 classes

In [3]:
import torch
import torch.nn as nn
import torchvision.models as models

def get_model(input_size: tuple = (3,32,32), #(224, 224, 3),
              hidden_size: int = 64,
              dropout: float = 0.5,
              num_classes: int = 10, #3,
              **kwargs
             ):
    # load the VGG16 network
    model = models.vgg16(pretrained=True)

    # freeze weights of base model
    for param in model.parameters():
        param.requires_grad = False
    
    # construct the head of the model 
    model.classifier[6] = nn.Sequential(
        nn.AvgPool2d(4),
        nn.Flatten(),
        nn.Linear(hidden_size,hidden_size),
        nn.BatchNorm2d(hidden_size),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(hidden_size, num_classes)
        )
    
    return model

In [4]:
model = get_model()
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

Find total parameters and trainable parameters

In [5]:
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

134,265,482 total parameters.
4,938 training parameters.


Load the CIFAR-10 dataset

In [6]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


Use the Adam optimizer and the softmax loss function

In [7]:
criterion = nn.Softmax()
optimizer = optim.Adam(params = model.parameters(), lr=0.001)

In [8]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        print(inputs)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        print(outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

tensor([[[[-0.0275, -0.0196,  0.0118,  ...,  0.1529,  0.1843,  0.2157],
          [ 0.2471,  0.2549,  0.2941,  ...,  0.1059,  0.1294,  0.1608],
          [ 0.5373,  0.5294,  0.5451,  ...,  0.1765,  0.1922,  0.1843],
          ...,
          [-0.3412, -0.4039, -0.5529,  ..., -0.2314, -0.1529, -0.0824],
          [-0.1373, -0.5373, -0.7412,  ..., -0.5373, -0.1686,  0.0980],
          [-0.2314, -0.3961, -0.4118,  ..., -0.0902, -0.0353,  0.0824]],

         [[ 0.1216,  0.1294,  0.1529,  ...,  0.3020,  0.3098,  0.3255],
          [ 0.3412,  0.3490,  0.3804,  ...,  0.2706,  0.2706,  0.2863],
          [ 0.5922,  0.5843,  0.5922,  ...,  0.3255,  0.3098,  0.3020],
          ...,
          [-0.3333, -0.4039, -0.5529,  ..., -0.2549, -0.1686, -0.1059],
          [-0.1373, -0.5373, -0.7333,  ..., -0.5843, -0.2000,  0.0510],
          [-0.2471, -0.4039, -0.4275,  ..., -0.1529, -0.0902,  0.0196]],

         [[ 0.3725,  0.3804,  0.3882,  ...,  0.5216,  0.5373,  0.5529],
          [ 0.4588,  0.4745,  

RuntimeError: non-empty 3D or 4D (batch mode) tensor expected for input